[辻真吾・矢吹太朗『ゼロからはじめるデータサイエンス入門』（講談社, 2021）](https://github.com/taroyabuki/fromzero)

In [ ]:
# Google Colaboratoryの環境設定
if (file.exists("/content")) {
  options(Ncpus = parallel::detectCores())
  installed_packages <- rownames(installed.packages())
  packages_to_install <- c("caret", "fable", "feasts", "prophet", "tsibble", "urca")
  install.packages(setdiff(packages_to_install, installed_packages))
  install.packages(c("ggplot2"))
}

## 12.1 日時と日時の列

In [ ]:
as.POSIXct("2021-01-01")

In [ ]:
library(tsibble)

seq(from = 2021, to = 2023, by = 1)

seq(from = yearmonth("202101"), to = yearmonth("202103"), by = 2)

seq(from = as.POSIXct("2021-01-01"), to = as.POSIXct("2021-01-03"), by = "1 day")

seq(from = as.POSIXct("2021-01-01 00:00:00"),
    to   = as.POSIXct("2021-01-01 03:00:00"), by = "2 hour")

## 12.2 時系列データの予測

In [ ]:
my_data <- as.vector(AirPassengers)

In [ ]:
n <- length(my_data) # データ数（144）
k <- 108             # 訓練データ数

In [ ]:
library(tidyverse)
library(tsibble)

my_ds <- seq(
  from = yearmonth("1949/01"),
  to   = yearmonth("1960/12"),
  by   = 1)
my_label <- rep(
  c("train", "test"),
  c(k, n - k))
my_df <- tsibble(
  ds    = my_ds,
  x     = 0:(n - 1),
  y     = my_data,
  label = my_label,
  index = ds) # 日時の列の指定

head(my_df)

In [ ]:
my_train <- my_df[  1:k , ]
my_test  <- my_df[-(1:k), ]
y <- my_test$y

In [ ]:
my_plot <- my_df %>%
  ggplot(aes(x = ds,
             y = y,
             color = label)) +
  geom_line()
my_plot

In [ ]:
library(caret)
my_lm_model <- train(form = y ~ x, data = my_train, method = "lm")
y_ <- my_lm_model %>% predict(my_test)
caret::RMSE(y, y_) # RMSE（テスト）

In [ ]:
y_ <- my_lm_model %>% predict(my_df)
tmp <- my_df %>%
  mutate(y = y_, label = "model")
my_plot + geom_line(data = tmp)

In [ ]:
library(fable)
my_arima_model <- my_train %>% model(ARIMA(y))
my_arima_model

In [ ]:
tmp <- my_arima_model %>% forecast(h = "3 years")
head(tmp)

In [ ]:
y_ <- tmp$.mean
caret::RMSE(y_, y)

In [ ]:
# 予測結果のみでよい場合
#tmp %>% autoplot

tmp %>% autoplot +
  geom_line(data = my_df,
            aes(x = ds,
                y = y,
                color = label))

In [ ]:
library(prophet)
my_prophet_model <- my_train %>%
  prophet(seasonality.mode = "multiplicative")

In [ ]:
tmp <- my_prophet_model %>% predict(my_test)
head(tmp[, c("ds", "yhat", "yhat_lower", "yhat_upper")])

In [ ]:
y_ <- tmp$yhat
caret::RMSE(y_, y)

In [ ]:
# my_prophet_model %>% plot(tmp) # 予測結果のみでよい場合

my_prophet_model %>% plot(tmp) +
  geom_line(data = my_train, aes(x = as.POSIXct(ds))) +
  geom_line(data = my_test,  aes(x = as.POSIXct(ds)), color = "red")